# Text Classification Tutorial
based on https://www.opencodez.com/python/text-classification-using-keras.htm

- You can choose between two Data Sets
    - Dataset ( http://archive.ics.uci.edu/ml/datasets/Reuter_50_50 ): contains documents of 50 different authors
    - Dataset ( http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz ): contains document of 20 different newsgroup genres

In [ ]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import itertools

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import metrics
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from sklearn.metrics import confusion_matrix
#from sklearn.feature_extraction.text import TfidfVectorizer
from pathlib import Path

## Retrieve Data

In [ ]:
# For reproducibility
np.random.seed(1237)
 
############ Newsgroup corpus ##############
#path_train = "./resources/20news-bydate/20news-bydate-train"
#path_test = "./resources/20news-bydate/20news-bydate-test"
#num_labels = 20
#test_files = [path_test + "/sci.med/59225",
#             path_test + "/sci.med/59229",
#             path_test + "/rec.autos/103007",
#             path_test + "/comp.graphics/38758",
#             path_test + "/comp.graphics/38777",]

############ Reuter corpus ############## 
path_train = "./resources/C50/C50train"
path_test = "./resources/C50/C50test"
num_labels = 50

test_files = [path_test + "/LydiaZajc/45801newsML.txt",
             path_test + "/LydiaZajc/377881newsML.txt",
            path_test + "/LydiaZajc/416661newsML.txt",
            path_test + "/FumikoFujisaki/416452newsML.txt",
            path_test + "/KouroshKarimkhany/357767newsML.txt"]


##########################

In [ ]:
files_train = skds.load_files(path_train,load_content=False)

file_paths = files_train.filenames #array with paths to every file
label_names = files_train.target_names #all labels
labelled_files_index = files_train.target #array with numerical represention of label for each file

In [ ]:
data_tags = ["filename","category","article"]
data_list = []

# Read and add data from file to a list
i=0
for f in file_paths:
    data_list.append((f,
                      label_names[labelled_files_index[i]],
                      Path(f).read_text()))
    i += 1
    

data = pd.DataFrame.from_records(data_list, columns=data_tags)

In [ ]:
data.head()

## Prepare Data

- we split the training data into 80/20
- each element contains content, tag(category) and file name


In [ ]:
#take 80% of data for training, the remaining 20% vor test/evaluation
train_size = int(len(data) * .8)
 
train_posts = data['article'][:train_size]
train_tags = data['category'][:train_size]
train_files_names = data['filename'][:train_size]
 
test_posts = data['article'][train_size:]
test_tags = data['category'][train_size:]
test_files_names = data['filename'][train_size:]

### Vectorization
- documents are modelled into tfidf vectors

In [ ]:
vocab_size = 15000 #vocabulary is restricted to 15000 words

tokenizer = Tokenizer(num_words=vocab_size)#takes the most [vocab_size] frequent words
tokenizer.fit_on_texts(train_posts)#selects features

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf') #weights features
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

### One Hot Encoding
- each label will be transformed in a binary vector

In [ ]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

#Testprint for label vector
print("Label: '" + train_tags[0] + "' will be transformed into:")
for i in range(num_labels):
    print(encoder.classes_[i] + ": " + str(y_train[0][i]))

## Model Data

fitting/training of the model with training(X) & test(Y) data

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(512)) 
model.add(Activation('sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

In [ ]:
batch_size = 100
epochs = 5
verbose = 1 #verbosity mode
validation_split = 0.1

model.compile(loss='categorical_crossentropy', #loss function for multiclass classification
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split)

### Evaluation

In [ ]:
result = model.evaluate(x_test, y_test,
                    batch_size=batch_size,
                    verbose=1)

print(model.metrics_names)
print(result)

## Predicting some unseen documents

- files are taken from the test folder
- steps: content tokenization, prediction, comparison with actual tag

In [ ]:
x_data = []
for t_f in test_files:
    t_f_data = Path(t_f).read_text()
    x_data.append(t_f_data)
 
x_data_series = pd.Series(x_data)
x_vectorized = tokenizer.texts_to_matrix(x_data_series, mode='tfidf')

In [ ]:
i=0
for x_t in x_vectorized:
    print(x_data[i][:80])
    prediction = model.predict(np.array([x_t])) #returns propabilities for each label
    predicted_label = encoder.classes_[np.argmax(prediction[0])] #chooses the most propable label
    print("File ->", test_files[i], "Predicted label: " + predicted_label)
    print("********************************")
    i += 1

## Evaluate on Test Data
### Retrieve Data

In [ ]:
files_test = skds.load_files(path_test,load_content=False)

In [ ]:
file_paths = files_test.filenames #array with paths to every file
label_names = files_test.target_names #all labels
labelled_files_index = files_test.target #array with numerical represention of label for each file

In [ ]:
data_tags = ["filename","category","article"]
data_list = []

# Read and add data from file to a list
i=0
for f in file_paths:
    data_list.append((f,
                      label_names[labelled_files_index[i]],
                      Path(f).read_text()))
    i += 1
    

data = pd.DataFrame.from_records(data_list, columns=data_tags)

### Prepare Data

In [ ]:
test_posts = data['article']
test_tags = data['category']
test_files_names = data['filename']

In [ ]:
#x_test = vectorizer.transform(test_posts)
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_test = encoder.transform(test_tags)

### Predict Labels

In [ ]:
probs = model.predict(x_test) #matrix with probabilities for each label per document
y_pred = np.argmax(probs, axis=1)#array with labels for each document

### Visualize Results

In [ ]:
cf = confusion_matrix(y_pred, labelled_files_index)

In [ ]:
labels = encoder.classes_

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
cax = ax.matshow(cf)
fig.colorbar(cax)
ax.set_yticks(np.arange(len(labels)))
ax.set_xticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation='vertical')
ax.set_yticklabels(labels)
#ax.set_yticklabels(np.arrage(len(test_tags)), test_tags)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()